# Health Insurance Risk Classifier
This project classifies individuals into health insurance risk categories based on their health metrics.
It uses a simple rule-based approach to categorize risk levels.

Author Carmine Giardino

Step 1:
Import necessary libraries and load the dataset


In [1]:
# Import pandas for data handling
import pandas as pd
# Import numpy for numerical operations
import numpy as np
# Import sqlite3 for database operations
import sqlite3

Step 2: read the health insurance data from a CSV file

In [2]:
path = './data/health_insurance_data.csv'
df = pd.read_csv(path)
print("Data loaded successfully.")
print(df.head())

Data loaded successfully.
    age     sex     bmi  children smoker     region      charges
0  19.0  female  27.900       0.0    yes  southwest  16884.92400
1  18.0    male  33.770       1.0     no  southeast   1725.55230
2  28.0    male  33.000       3.0     no  southeast   4449.46200
3  33.0    male  22.705       0.0     no  northwest  21984.47061
4  32.0    male  28.880       0.0     no  northwest   3866.85520


Step 3: clean the data by handling missing values

In [3]:
# Fill missing values in numeric columns with mean
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html
for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].fillna(df[col].mean())

# Fill missing values in non-numeric columns with mode
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.mode.html
for col in df.select_dtypes(exclude=[np.number]).columns:
    mode = df[col].mode()[0] if not df[col].mode().empty else None
    df[col] = df[col].fillna(mode)

print("Missing values handled.")
print(df.isnull().sum())

Missing values handled.
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


Step 3.5: encode categorical variables

In [4]:
# Encode categorical variables for analysis
# sex: male=1, female=0
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html
df['sex_encoded'] = df['sex'].map({'male': 1, 'female': 0})

# smoker: yes=1, no=0
df['smoker_encoded'] = df['smoker'].map({'yes': 1, 'no': 0})

# region: use one-hot encoding
# One-hot encoding converts categorical text into a numerical format without implying any ordinal relationship between regions.
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
region_dummies = pd.get_dummies(df['region'], prefix='region')
df = pd.concat([df, region_dummies], axis=1)

print("Categorical variables encoded.")
print(df.head())
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

Categorical variables encoded.
    age     sex     bmi  children smoker     region      charges  sex_encoded  \
0  19.0  female  27.900       0.0    yes  southwest  16884.92400            0   
1  18.0    male  33.770       1.0     no  southeast   1725.55230            1   
2  28.0    male  33.000       3.0     no  southeast   4449.46200            1   
3  33.0    male  22.705       0.0     no  northwest  21984.47061            1   
4  32.0    male  28.880       0.0     no  northwest   3866.85520            1   

   smoker_encoded  region_northeast  region_northwest  region_southeast  \
0               1             False             False             False   
1               0             False             False              True   
2               0             False             False              True   
3               0             False              True             False   
4               0             False              True             False   

   region_southwest  
0        

Step 4: normalize numeric features for better comparison

In [5]:
# Preserve original charges for later correlation analysis
df['charges_original'] = df['charges'].copy()

# Identify numeric columns
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html
numeric_cols = df.select_dtypes(include=[np.number]).columns
# Exclude charges_original from normalization
numeric_cols = [col for col in numeric_cols if col != 'charges_original']

# Normalize using manual min-max scaling
for col in numeric_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    df[col] = (df[col] - min_val) / (max_val - min_val)
print("Numeric features normalized.")
print(df.head())

Numeric features normalized.
        age     sex       bmi  children smoker     region   charges  \
0  0.021739  female  0.321227       0.0    yes  southwest  0.251611   
1  0.000000    male  0.479150       0.2     no  southeast  0.009636   
2  0.217391    male  0.458434       0.6     no  southeast  0.053115   
3  0.326087    male  0.181464       0.0     no  northwest  0.333010   
4  0.304348    male  0.347592       0.0     no  northwest  0.043816   

   sex_encoded  smoker_encoded  region_northeast  region_northwest  \
0          0.0             1.0             False             False   
1          1.0             0.0             False             False   
2          1.0             0.0             False             False   
3          1.0             0.0             False              True   
4          1.0             0.0             False              True   

   region_southeast  region_southwest  charges_original  
0             False              True       16884.92400  
1      

Step 4.5: save the data to SQLite database

In [6]:
# Create SQLite database connection
db_path = './data/health_insurance.db'
# Connect to SQLite database (it will be created if it doesn't exist)
# Ref: https://docs.python.org/3/library/sqlite3.html#sqlite3.connect
conn = sqlite3.connect(db_path)

# Save the processed data to SQLite
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
df.to_sql('health_insurance_processed', conn, if_exists='replace', index=False)

# Also save the original data (before normalization) if needed
df_original = pd.read_csv(path)
df_original.to_sql('health_insurance_original', conn, if_exists='replace', index=False)

# Close the connection
# Ref: https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.close
conn.close()

print(f"Data saved to SQLite database at: {db_path}")
print("Tables created: 'health_insurance_original' and 'health_insurance_processed'")

Data saved to SQLite database at: ./data/health_insurance.db
Tables created: 'health_insurance_original' and 'health_insurance_processed'


Step 5: define risk classification rules based on health metrics

In [7]:
# The function assigns weights to different factors to compute a risk score and classifies into 'Low', 'Medium', 'High' risk categories.
def classify_risk(row):
    risk_score = 0
    
    # Age scoring - higher impact for older ages
    if row['age'] > 0.7:
        risk_score += 4
    elif row['age'] > 0.4:
        risk_score += 2
    elif row['age'] > 0.2:
        risk_score += 1
    
    # BMI scoring - higher impact for obesity
    if row['bmi'] > 0.7:
        risk_score += 4
    elif row['bmi'] > 0.5:
        risk_score += 2
    elif row['bmi'] > 0.3:
        risk_score += 1
    
    # Smoker - very high impact
    if row['smoker'] == 'yes':
        risk_score += 5
    
    # Children - lower impact
    if row['children'] >= 3:
        risk_score += 1
    elif row['children'] >= 1:
        risk_score += 0.5
    
    # Adjusted thresholds for balanced classification
    if risk_score >= 7:
        return 'High'
    elif risk_score >= 4:
        return 'Medium'
    else:
        return 'Low'


Step 6: apply risk classification to all records

In [8]:
# Apply the risk classification function to each row
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
df['risk_category'] = df.apply(classify_risk, axis=1)

print("Risk classification applied.")
print(df[['age', 'bmi', 'smoker', 'children', 'risk_category']].head(10))
print(f"\nRisk category distribution:")
print(df['risk_category'].value_counts())
print(f"\nPercentage distribution:")
print(df['risk_category'].value_counts(normalize=True) * 100)

Risk classification applied.
        age       bmi smoker  children risk_category
0  0.021739  0.321227    yes  0.000000        Medium
1  0.000000  0.479150     no  0.200000           Low
2  0.217391  0.458434     no  0.600000           Low
3  0.326087  0.181464     no  0.000000           Low
4  0.304348  0.347592     no  0.000000           Low
5  0.282609  0.263115     no  0.000000           Low
6  0.608696  0.470272     no  0.200000           Low
7  0.413043  0.316922     no  0.600000           Low
8  0.413043  0.373150     no  0.400000           Low
9  0.913043  0.265806     no  0.219147        Medium

Risk category distribution:
risk_category
Low       686
Medium    443
High      209
Name: count, dtype: int64

Percentage distribution:
risk_category
Low       51.270553
Medium    33.109118
High      15.620329
Name: proportion, dtype: float64


Step 7: save the final dataset with risk classifications to SQLite database

In [9]:
# Reconnect to SQLite database
conn = sqlite3.connect(db_path)

# Save the final dataset with risk classifications
df.to_sql('health_insurance_with_risk', conn, if_exists='replace', index=False)

print(f"Dataset with risk classifications saved to 'health_insurance_with_risk' table")
print(f"Total records: {len(df)}")

Dataset with risk classifications saved to 'health_insurance_with_risk' table
Total records: 1338


Step 8: run SQL queries to confirm correct storage and retrieval

In [10]:
# Query 1: Count total records
query1 = "SELECT COUNT(*) as total_records FROM health_insurance_with_risk"
# Run the query and fetch results into a DataFrame
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html
result1 = pd.read_sql_query(query1, conn)
print("Query 1 - Total records:")
print(result1)
print()

# Query 2: Count by risk category
# Group by risk category and count occurrences
# Ref: https://www.sqltutorial.org/sql-group-by/
query2 = """
SELECT risk_category, COUNT(*) as count 
FROM health_insurance_with_risk 
GROUP BY risk_category 
ORDER BY count DESC
"""
result2 = pd.read_sql_query(query2, conn)
print("Query 2 - Risk category distribution:")
print(result2)
print()

# Query 3: Average age and BMI by risk category
# Ref: https://www.sqltutorial.org/sql-aggregate-functions/sql-avg/
query3 = """
SELECT risk_category, 
       AVG(age) as avg_age, 
       AVG(bmi) as avg_bmi,
       COUNT(*) as count
FROM health_insurance_with_risk 
GROUP BY risk_category
"""
result3 = pd.read_sql_query(query3, conn)
print("Query 3 - Average age and BMI by risk category:")
print(result3)
print()

# Query 4: Sample records from each risk category
query4 = """
SELECT risk_category, age, bmi, smoker, children, charges_original
FROM health_insurance_with_risk
ORDER BY risk_category, charges_original DESC
LIMIT 15
"""
result4 = pd.read_sql_query(query4, conn)
print("Query 4 - Sample records from database:")
print(result4)

# Close the connection
conn.close()
print("\nSQL queries completed successfully. Database connection closed.")



Query 1 - Total records:
   total_records
0           1338

Query 2 - Risk category distribution:
  risk_category  count
0           Low    686
1        Medium    443
2          High    209

Query 3 - Average age and BMI by risk category:
  risk_category   avg_age   avg_bmi  count
0          High  0.571770  0.450128    209
1           Low  0.297415  0.351992    686
2        Medium  0.662921  0.437414    443

Query 4 - Sample records from database:
   risk_category       age       bmi smoker  children  charges_original
0           High  0.782609  0.846112    yes       0.0       63770.42801
1           High  0.586957  0.387409    yes       0.0       62592.87309
2           High  0.739130  0.498386    yes       0.6       60021.39897
3           High  0.282609  0.595507    yes       0.2       58571.07448
4           High  0.326087  0.526500    yes       0.0       55135.40209
5           High  0.913043  0.453054    yes       0.0       52590.82939
6           High  0.217391  0.549906    yes 

# End